# Logistic regression

## Theory

Logistic regression is a classification algorithm. Here, it will be used as a binary classifier, but it can also be expanded to multi-class classification.
Most of the time the classes are labeled as zero and one. For example, in the example below malignant breast cancer cells are class one and benign breast cancer cells 
correspond to class zero. A linear combination of the features is still used like in linear regression. However, linear regression cannot be used as its hypothesis 
has a codomain of $\cal{R}$, but $y_i$ can only have two values (zero or one) in logistic regression. This means that Logistic regression needs a function whose 
codomain is $(0, 1)$. A function that satisfies this constraint is the sigmoid function

$$ h_\theta (x_i) = \frac{1}{1+e^{\theta x_i}}$$

In the exponential, the hypothesis of linear regression is found. The objective function of logistic regression is the log-likelihood:

$$ - \frac{1}{N} \sum_i y_i \ln\left( h_\theta(x_i) \right) + \left( 1 - y_i \right) \ln\left(1-h_\theta(x_i) \right)$$

Gradient descent is again used to optimize the objective function, using the same formula as linear regression.

$$
\theta^{(j)} \leftarrow \theta^{(j)} - \alpha \frac{1}{N} \sum_{i=0}^N \left( h_{\pmb{\theta}} \left( \pmb{x}_i \right) - y_i \right ) x_i^{(j)}
$$


## Implementation

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def sigmoid(X, theta):
    return 1/(1+np.exp(-X@theta))

In [3]:
def loglikelihood(theta, X, y):
    return -np.sum(y*np.log(sigmoid(X, theta)) + (1-y)*np.log(1-sigmoid(X, theta)))/X.shape[0]

In [4]:
df = pd.read_csv("../data/BreastCancer.csv")
df.drop(labels=["id", "Unnamed: 32"], axis=1, inplace=True)

# Change the labels M and B to 1 and 0 respectively
df.replace("M", 1, inplace=True)
df.replace("B", 0, inplace=True)

In [5]:
# Create feature vector X and class vector y
X = df.drop(labels=['diagnosis'], axis=1).to_numpy()
y = df['diagnosis'].to_numpy()

# Split the data into a 80% training and 20% test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
def gradient_descent(X_train, y_train, learning_rate, num_iterations, threshold=1e-3, blog=False):
    """
    Use gradient descent to optimize regression parameters theta in order to find the best straight
    line for the given points

    :param points: points to fit the line
    :param learning_rate: learning rate used in the algorithm
    :param num_iterations: maximum number of iterations
    :param threshold: minimum difference between two sequential mean squared
        error values (default is 1e-3)
    :param blog: indicates to print the cost at every step (default is False)
    """

    # Init values
    theta = np.zeros(X_train.shape[1])
    m = X_train.shape[0]
    iteration = 0

    J = loglikelihood(theta, X_train, y_train)
    prev_J = np.inf

    # Loop until convergence or maximum number of iterations is reached
    while iteration < num_iterations and np.all(np.abs(J - prev_J) > threshold):

        new_thetas = np.zeros(len(theta))
        prev_J = J

        # Compute new theta's using the gradient
        for j, theta_j in enumerate(theta):
            new_thetas[j] = theta_j - learning_rate / m * np.sum(
                (sigmoid(X_train, theta) - y_train) * X_train[:, j]
            )

        theta = new_thetas
        # Compute new MSE
        J = loglikelihood(theta, X_train, y_train)

        if blog:
            print(f"{iteration}: {J}")

        iteration += 1

    return theta, J


In [7]:
X_train_scaled = StandardScaler().fit_transform(X_train)

theta_opt, J = gradient_descent(X_train_scaled, y_train, 1, 1e3, blog=True)

0: 0.1765562847799052
1: 0.14168152486288937
2: 0.12597680293130076
3: 0.11865746865556694
4: 0.11392830594814492
5: 0.11022378970399437
6: 0.10712659074630146
7: 0.10446078750151873
8: 0.10212653791773602
9: 0.10005751186805145
10: 0.0982058485829798
11: 0.09653536682323952
12: 0.09501790144512029
13: 0.09363107256453192
14: 0.09235681553594174
15: 0.09118036073870096
16: 0.09008949998152911
17: 0.08907404528814196
18: 0.08812542167092427


In [12]:
X_test_scaled = StandardScaler().fit_transform(X_test)

y_pred = [0 if h < 0.5 else 1 for h in sigmoid(X_test_scaled, theta_opt)]
accuracy = np.sum(y_pred == y_test)/y_test.shape[0]

print(accuracy)

0.9824561403508771
